# IMPORTAÇÃO DE BIBLIOTECAS

In [281]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import random
import statistics
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

In [282]:
# COMMON VARIABLES

alg_to_analyze = [
    'vader_sentiment_analysis_score',
    'vader_sentiment_analysis_polarity'
]

tpd_columns = ["text", "created_at", "id_user", "vader_sentiment_analysis_score", "vader_sentiment_analysis_polarity",
                         "oplexicon_sentiment_analysis_score", "oplexicon_sentiment_analysis_polarity",
                         "sentistrength_sentiment_analysis_score", "sentistrength_sentiment_analysis_polarity",
                         "sentilexpt_sentiment_analysis_score", "sentilexpt_sentiment_analysis_polarity",
                         "final_score", "final_polarity"]

# LEITURA DE ARQUIVOS

In [283]:
!python3 '/Users/leosilva/development/thesis/workspacePyCharm/TwitterDataMining/main_generate_json_files.py' --all_tweets True

Starting script...
Removing existing file...
Ending script...


In [284]:
data = pd.read_json(r'/Users/leosilva/development/thesis/workspacePyCharm/TwitterDataMining/alldata.json')
data.head()

,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
id_str_twitter,23781146,15148309,927028057763995648,77648005,3429765093,10794662,1034514325644431361,529445600,12050562,62314491,18289937,172880784,18609547,2164986296,1026783544457273345,109604376,1040939562774790144,69734383,119897261
name,1,2,3,None,4,5,6,None,7,8,None,9,10,11,12,13,14,15,16
screen_name,Fe Rebelatto,Sommelier de Airbnb,✪ Felippe,Conference Distinguished Specialist,Paula Santana,Maharaja of the Legacies,Jessica,"var nickname = ""Riquettinha"";",Roberta Arcoverde,Gabriela,Thamara Andrade,felipe.ts ⚡,Juan Lopes,Rafael Dantas,iagor com i,T̷o̷m̷o̷y̷o̷ ̷ F̷u̷r̷a̷c̷ã̷o̷,Esdras Xavier,belle-🏴.dev || twitch.tv/bellesamways,Willian Justen
location,rebelatto,Guilh_rm_,FelippeRegazio,NannoKa,psanrosa13,rponte,dev_jessi,Riquettinha,rla4,Gabrielathalita,thamyk,felipefialho_,juanplopes,RaffaelDantass,Iagor51,RafaelMansilha,esdras_xavieer,bellesamways,Willian_justen
url,"Rio de Janeiro, Brasil",somewhere in Brazil,,Ctba-PR Brasil,Brasil,"Ceará, Brazil",São Paulo - Brasil,"Belo Horizonte, Brasil","Sao Paulo, Brazil",Belo Horizonte,"Belo Horizonte, Brazil.",Dark Side of the Moon,"Rio de Janeiro, Brasil","São Paulo, Brasil",Pvh,"Palmas, Tocantins, Brasil","Jaraguá do Sul, Brasil",Curitiba,"Petrópolis, Brasil"


In [285]:
data_psi = pd.read_csv(r'/Users/leosilva/Documents/Estudo/Doutorado/Coimbra/2019-2020/Disciplinas/Thesis/analises_psicologos/Tweets_consolidados.csv')
data_psi.head()

,ID,Classificacao_PSI1,Classificacao_PSI2,Classificacao_PSI3
0,102600,Negativo Fraco,Positivo Fraco,Neutro
1,102895,Negativo Fraco,Negativo Forte,Negativo Forte
2,103085,Negativo Forte,Negativo Forte,Negativo Forte
3,103092,Positivo Fraco,Positivo Fraco,Positivo Fraco
4,103200,Positivo Fraco,Negativo Fraco,Negativo Forte


In [286]:
data_participants = pd.read_csv(r'/Users/leosilva/Documents/Estudo/Doutorado/Coimbra/2019-2020/Disciplinas/Thesis/tweets_dev/classification/tweets_consolidados.csv')
data_participants.head()

,id,classificacao,Id_usuario
0,197052,Positivo Fraco,43
1,197097,Neutro,43
2,198955,Neutro,43
3,200154,Positivo Fraco,43
4,200645,Positivo Fraco,43


## Preparação do DataFrame das respostas das psicólogas

In [287]:
# create a list of our conditions
conditions = [
    (data_psi['Classificacao_PSI1'] == 'Positivo Fraco'),
    (data_psi['Classificacao_PSI1'] == 'Positivo Forte'),
    (data_psi['Classificacao_PSI1'] == 'Neutro'),
    (data_psi['Classificacao_PSI1'] == 'Negativo Fraco'),
    (data_psi['Classificacao_PSI1'] == 'Negativo Forte')
    ]

# create a list of the values we want to assign for each condition
values = ['pos', 'pos', 'neu', 'neg', 'neg']

# create a new column and use np.select to assign values to it using our lists as arguments
data_psi['Classificacao_PSI1_3_cat'] = np.select(conditions, values)

data_psi.head()

,ID,Classificacao_PSI1,Classificacao_PSI2,Classificacao_PSI3,Classificacao_PSI1_3_cat
0,102600,Negativo Fraco,Positivo Fraco,Neutro,neg
1,102895,Negativo Fraco,Negativo Forte,Negativo Forte,neg
2,103085,Negativo Forte,Negativo Forte,Negativo Forte,neg
3,103092,Positivo Fraco,Positivo Fraco,Positivo Fraco,pos
4,103200,Positivo Fraco,Negativo Fraco,Negativo Forte,pos


In [288]:
# create a list of our conditions
conditions = [
    (data_psi['Classificacao_PSI2'] == 'Positivo Fraco'),
    (data_psi['Classificacao_PSI2'] == 'Positivo Forte'),
    (data_psi['Classificacao_PSI2'] == 'Neutro'),
    (data_psi['Classificacao_PSI2'] == 'Negativo Fraco'),
    (data_psi['Classificacao_PSI2'] == 'Negativo Forte')
    ]

# create a list of the values we want to assign for each condition
values = ['pos', 'pos', 'neu', 'neg', 'neg']

# create a new column and use np.select to assign values to it using our lists as arguments
data_psi['Classificacao_PSI2_3_cat'] = np.select(conditions, values)

data_psi.head()

,ID,Classificacao_PSI1,Classificacao_PSI2,Classificacao_PSI3,Classificacao_PSI1_3_cat,Classificacao_PSI2_3_cat
0,102600,Negativo Fraco,Positivo Fraco,Neutro,neg,pos
1,102895,Negativo Fraco,Negativo Forte,Negativo Forte,neg,neg
2,103085,Negativo Forte,Negativo Forte,Negativo Forte,neg,neg
3,103092,Positivo Fraco,Positivo Fraco,Positivo Fraco,pos,pos
4,103200,Positivo Fraco,Negativo Fraco,Negativo Forte,pos,neg


In [289]:
# create a list of our conditions
conditions = [
    (data_psi['Classificacao_PSI3'] == 'Positivo Fraco'),
    (data_psi['Classificacao_PSI3'] == 'Positivo Forte'),
    (data_psi['Classificacao_PSI3'] == 'Neutro'),
    (data_psi['Classificacao_PSI3'] == 'Negativo Fraco'),
    (data_psi['Classificacao_PSI3'] == 'Negativo Forte')
    ]

# create a list of the values we want to assign for each condition
values = ['pos', 'pos', 'neu', 'neg', 'neg']

# create a new column and use np.select to assign values to it using our lists as arguments
data_psi['Classificacao_PSI3_3_cat'] = np.select(conditions, values)

data_psi.head()

,ID,Classificacao_PSI1,Classificacao_PSI2,Classificacao_PSI3,Classificacao_PSI1_3_cat,Classificacao_PSI2_3_cat,Classificacao_PSI3_3_cat
0,102600,Negativo Fraco,Positivo Fraco,Neutro,neg,pos,neu
1,102895,Negativo Fraco,Negativo Forte,Negativo Forte,neg,neg,neg
2,103085,Negativo Forte,Negativo Forte,Negativo Forte,neg,neg,neg
3,103092,Positivo Fraco,Positivo Fraco,Positivo Fraco,pos,pos,pos
4,103200,Positivo Fraco,Negativo Fraco,Negativo Forte,pos,neg,neg


In [290]:
data_psi.columns = ['id',
                    'Class_PSI1_5_cat',
                    'Class_PSI2_5_cat',
                    'Class_PSI3_5_cat',
                    'Class_PSI1_3_cat',
                    'Class_PSI2_3_cat',
                    'Class_PSI3_3_cat']

data_psi.head()

,id,Class_PSI1_5_cat,Class_PSI2_5_cat,Class_PSI3_5_cat,Class_PSI1_3_cat,Class_PSI2_3_cat,Class_PSI3_3_cat
0,102600,Negativo Fraco,Positivo Fraco,Neutro,neg,pos,neu
1,102895,Negativo Fraco,Negativo Forte,Negativo Forte,neg,neg,neg
2,103085,Negativo Forte,Negativo Forte,Negativo Forte,neg,neg,neg
3,103092,Positivo Fraco,Positivo Fraco,Positivo Fraco,pos,pos,pos
4,103200,Positivo Fraco,Negativo Fraco,Negativo Forte,pos,neg,neg


In [291]:
# create a list of our conditions
conditions = [
    (data_psi['Class_PSI1_5_cat'] == 'Positivo Fraco'),
    (data_psi['Class_PSI1_5_cat'] == 'Positivo Forte'),
    (data_psi['Class_PSI1_5_cat'] == 'Neutro'),
    (data_psi['Class_PSI1_5_cat'] == 'Negativo Fraco'),
    (data_psi['Class_PSI1_5_cat'] == 'Negativo Forte')
    ]

# create a list of the values we want to assign for each condition
values = ['w_pos', 's_pos', 'neu', 'w_neg', 's_neg']

# create a new column and use np.select to assign values to it using our lists as arguments
data_psi['Class_PSI1_5_cat'] = np.select(conditions, values)

data_psi.head()

,id,Class_PSI1_5_cat,Class_PSI2_5_cat,Class_PSI3_5_cat,Class_PSI1_3_cat,Class_PSI2_3_cat,Class_PSI3_3_cat
0,102600,w_neg,Positivo Fraco,Neutro,neg,pos,neu
1,102895,w_neg,Negativo Forte,Negativo Forte,neg,neg,neg
2,103085,s_neg,Negativo Forte,Negativo Forte,neg,neg,neg
3,103092,w_pos,Positivo Fraco,Positivo Fraco,pos,pos,pos
4,103200,w_pos,Negativo Fraco,Negativo Forte,pos,neg,neg


In [292]:
# create a list of our conditions
conditions = [
    (data_psi['Class_PSI2_5_cat'] == 'Positivo Fraco'),
    (data_psi['Class_PSI2_5_cat'] == 'Positivo Forte'),
    (data_psi['Class_PSI2_5_cat'] == 'Neutro'),
    (data_psi['Class_PSI2_5_cat'] == 'Negativo Fraco'),
    (data_psi['Class_PSI2_5_cat'] == 'Negativo Forte')
    ]

# create a list of the values we want to assign for each condition
values = ['w_pos', 's_pos', 'neu', 'w_neg', 's_neg']

# create a new column and use np.select to assign values to it using our lists as arguments
data_psi['Class_PSI2_5_cat'] = np.select(conditions, values)

data_psi.head()

,id,Class_PSI1_5_cat,Class_PSI2_5_cat,Class_PSI3_5_cat,Class_PSI1_3_cat,Class_PSI2_3_cat,Class_PSI3_3_cat
0,102600,w_neg,w_pos,Neutro,neg,pos,neu
1,102895,w_neg,s_neg,Negativo Forte,neg,neg,neg
2,103085,s_neg,s_neg,Negativo Forte,neg,neg,neg
3,103092,w_pos,w_pos,Positivo Fraco,pos,pos,pos
4,103200,w_pos,w_neg,Negativo Forte,pos,neg,neg


In [293]:
# create a list of our conditions
conditions = [
    (data_psi['Class_PSI3_5_cat'] == 'Positivo Fraco'),
    (data_psi['Class_PSI3_5_cat'] == 'Positivo Forte'),
    (data_psi['Class_PSI3_5_cat'] == 'Neutro'),
    (data_psi['Class_PSI3_5_cat'] == 'Negativo Fraco'),
    (data_psi['Class_PSI3_5_cat'] == 'Negativo Forte')
    ]

# create a list of the values we want to assign for each condition
values = ['w_pos', 's_pos', 'neu', 'w_neg', 's_neg']

# create a new column and use np.select to assign values to it using our lists as arguments
data_psi['Class_PSI3_5_cat'] = np.select(conditions, values)

data_psi.head()

,id,Class_PSI1_5_cat,Class_PSI2_5_cat,Class_PSI3_5_cat,Class_PSI1_3_cat,Class_PSI2_3_cat,Class_PSI3_3_cat
0,102600,w_neg,w_pos,neu,neg,pos,neu
1,102895,w_neg,s_neg,s_neg,neg,neg,neg
2,103085,s_neg,s_neg,s_neg,neg,neg,neg
3,103092,w_pos,w_pos,w_pos,pos,pos,pos
4,103200,w_pos,w_neg,s_neg,pos,neg,neg


In [294]:
print(len(data_psi))

455


## Preparação do DataFrame das respostas dos participantes

In [295]:
# create a list of our conditions
conditions = [
    (data_participants['classificacao'] == 'Positivo Fraco'),
    (data_participants['classificacao'] == 'Positivo Forte'),
    (data_participants['classificacao'] == 'Neutro'),
    (data_participants['classificacao'] == 'Negativo Fraco'),
    (data_participants['classificacao'] == 'Negativo Forte')
    ]

# create a list of the values we want to assign for each condition
values = ['pos', 'pos', 'neu', 'neg', 'neg']

# create a new column and use np.select to assign values to it using our lists as arguments
data_participants['classificacao_3_cat'] = np.select(conditions, values)

data_participants.head()

,id,classificacao,Id_usuario,classificacao_3_cat
0,197052,Positivo Fraco,43,pos
1,197097,Neutro,43,neu
2,198955,Neutro,43,neu
3,200154,Positivo Fraco,43,pos
4,200645,Positivo Fraco,43,pos


In [296]:
# create a list of our conditions
conditions = [
    (data_participants['classificacao'] == 'Positivo Fraco'),
    (data_participants['classificacao'] == 'Positivo Forte'),
    (data_participants['classificacao'] == 'Neutro'),
    (data_participants['classificacao'] == 'Negativo Fraco'),
    (data_participants['classificacao'] == 'Negativo Forte')
    ]

# create a list of the values we want to assign for each condition
values = ['w_pos', 's_pos', 'neu', 'w_neg', 's_neg']

# create a new column and use np.select to assign values to it using our lists as arguments
data_participants['classificacao_5_cat'] = np.select(conditions, values)

data_participants.head()

,id,classificacao,Id_usuario,classificacao_3_cat,classificacao_5_cat
0,197052,Positivo Fraco,43,pos,w_pos
1,197097,Neutro,43,neu,neu
2,198955,Neutro,43,neu,neu
3,200154,Positivo Fraco,43,pos,w_pos
4,200645,Positivo Fraco,43,pos,w_pos


In [297]:
print(data_psi['Class_PSI3_5_cat'].value_counts())

neu      306
w_pos     61
w_neg     44
s_neg     25
s_pos     19
Name: Class_PSI3_5_cat, dtype: int64


In [298]:
tweets = []

for d in data:
    if data[d]['bigfive']:
        tweets.append(data[d]['tweets'])

final_tweets = []
for i in tweets:
    final_tweets.extend(i)

In [299]:
print(len(final_tweets))

91632


In [300]:
def fill_manual_polarities_3_cat(df):
    for d in df['id']:
        pol = []
        t = df.query('id == {}'.format(d))
        pol.append(t['Class_PSI1_3_cat'].values[0])
        pol.append(t['Class_PSI2_3_cat'].values[0])
        pol.append(t['Class_PSI3_3_cat'].values[0])
        c = dict(Counter(pol))
        for i in c.items():
            if i[1] >= 2:
                df.loc[df['id'] == d, 'manual_polarity_3_cat'] = i[0]
    
    return df

In [301]:
def fill_manual_polarities_5_cat(df):
    for d in df['id']:
        pol = []
        t = df.query('id == {}'.format(d))
        pol.append(t['Class_PSI1_5_cat'].values[0])
        pol.append(t['Class_PSI2_5_cat'].values[0])
        pol.append(t['Class_PSI3_5_cat'].values[0])
        c = dict(Counter(pol))
        for i in c.items():
            if i[1] >= 2:
                df.loc[df['id'] == d, 'manual_polarity_5_cat'] = i[0]
    return df

In [302]:
def conf_matrix(x, col1, col2):
    if x[col1] == 1 and x[col2] == 1:
        return 'TP'
    elif x[col1] == 1 and x[col2] == -1:
        return 'FN'
    elif x[col1] == -1 and x[col2] == 1:
        return 'FP'
    elif x[col1] == -1 and x[col2] == -1:
        return 'TN'
    else:
        return 0

In [303]:
rdf = pd.DataFrame(columns=['Sentiment Lexicon', 'Accuracy', 'Precision - Positive', 
                                'Recall - Positive', 'F1-Score - Positive',
                                'Precision - Negative', 
                                'Recall - Negative', 'F1-Score - Negative'
                               ])

In [304]:
data_psi.head()

,id,Class_PSI1_5_cat,Class_PSI2_5_cat,Class_PSI3_5_cat,Class_PSI1_3_cat,Class_PSI2_3_cat,Class_PSI3_3_cat
0,102600,w_neg,w_pos,neu,neg,pos,neu
1,102895,w_neg,s_neg,s_neg,neg,neg,neg
2,103085,s_neg,s_neg,s_neg,neg,neg,neg
3,103092,w_pos,w_pos,w_pos,pos,pos,pos
4,103200,w_pos,w_neg,s_neg,pos,neg,neg


In [305]:
df_data_psi_3_cat = fill_manual_polarities_3_cat(data_psi)
df_data_psi_3_cat = df_data_psi_3_cat.dropna()
df_data_psi_3_cat.head()

,id,Class_PSI1_5_cat,Class_PSI2_5_cat,Class_PSI3_5_cat,Class_PSI1_3_cat,Class_PSI2_3_cat,Class_PSI3_3_cat,manual_polarity_3_cat
1,102895,w_neg,s_neg,s_neg,neg,neg,neg,neg
2,103085,s_neg,s_neg,s_neg,neg,neg,neg,neg
3,103092,w_pos,w_pos,w_pos,pos,pos,pos,pos
4,103200,w_pos,w_neg,s_neg,pos,neg,neg,neg
6,103440,w_pos,s_neg,w_neg,pos,neg,neg,neg


In [306]:
df_data_psi_5_cat = fill_manual_polarities_5_cat(data_psi)
df_data_psi_5_cat.head()

,id,Class_PSI1_5_cat,Class_PSI2_5_cat,Class_PSI3_5_cat,Class_PSI1_3_cat,Class_PSI2_3_cat,Class_PSI3_3_cat,manual_polarity_3_cat,manual_polarity_5_cat
0,102600,w_neg,w_pos,neu,neg,pos,neu,NaN,NaN
1,102895,w_neg,s_neg,s_neg,neg,neg,neg,neg,s_neg
2,103085,s_neg,s_neg,s_neg,neg,neg,neg,neg,s_neg
3,103092,w_pos,w_pos,w_pos,pos,pos,pos,pos,w_pos
4,103200,w_pos,w_neg,s_neg,pos,neg,neg,neg,NaN


In [307]:
df_data_psi_5_cat = df_data_psi_5_cat.dropna()
df_data_psi_5_cat.head()

,id,Class_PSI1_5_cat,Class_PSI2_5_cat,Class_PSI3_5_cat,Class_PSI1_3_cat,Class_PSI2_3_cat,Class_PSI3_3_cat,manual_polarity_3_cat,manual_polarity_5_cat
1,102895,w_neg,s_neg,s_neg,neg,neg,neg,neg,s_neg
2,103085,s_neg,s_neg,s_neg,neg,neg,neg,neg,s_neg
3,103092,w_pos,w_pos,w_pos,pos,pos,pos,pos,w_pos
8,103546,s_pos,s_pos,w_pos,pos,pos,pos,pos,s_pos
10,103823,w_neg,w_neg,neu,neg,neg,neu,neg,w_neg


In [308]:
print(len(df_data_psi_5_cat))

350


In [309]:
def create_psycho_versus_participants_df(df, category):

    if category == 3:
        fields = {
            'd': ['manual_polarity_3_cat', 'participants_polarity_3_cat'],
            'r': ['manual_polarity_3_cat', 'classificacao_3_cat']
        }
    elif category == 5:
        fields = {
            'd': ['manual_polarity_5_cat', 'participants_polarity_5_cat'],
            'r': ['manual_polarity_5_cat', 'classificacao_5_cat']
        }


    d = {
        'id':[],
        fields['d'][0]:[],
        fields['d'][1]:[]
    }

    ids_not_found = []

    count = 0
    for idx in data_participants['id']:
        found = False
        for i in df['id']:
            if idx == i:
                found = True
                count = count + 1
                t = df.query('id == {}'.format(i))
                d['id'].append(t[t.keys()[0]].values[0])
                d[fields['d'][0]].append(t[fields['r'][0]].tolist()[0])
                p_t = data_participants.loc[data_participants['id'] == idx]
                d[fields['d'][1]].append(p_t[fields['r'][1]].tolist()[0])
        if found == False:
            ids_not_found.append(idx)

    print("ids not found: ", len(ids_not_found))
    temp_pd = pd.DataFrame(d)
    return temp_pd

In [310]:
def perform_statistics_between_psycho_participants(df, cat, prefix):
    if cat == 3:
        df['participants_prediction'] = df['participants_polarity_3_cat'].apply(lambda x: -1 if x == 'neg' else (1 if x == 'pos' else 0))
        df['psychologists_prediction'] = df['manual_polarity_3_cat'].apply(lambda x: -1 if x == 'neg' else (1 if x == 'pos' else 0))
    elif cat == 5:
        if prefix == 'wp':
            df['participants_prediction'] = df['participants_polarity_5_cat'].apply(lambda x: -1 if x == 'w_neg' or x == 's_neg' else (1 if x == 'w_pos' else 0))
            df['psychologists_prediction'] = df['manual_polarity_5_cat'].apply(lambda x: -1 if x == 'w_neg' or x == 's_neg' else (1 if x == 'w_pos' else 0))
        elif prefix == 'sp':
            df['participants_prediction'] = df['participants_polarity_5_cat'].apply(lambda x: -1 if x == 's_neg' or x == 'w_neg' else (1 if x == 's_pos' else 0))
            df['psychologists_prediction'] = df['manual_polarity_5_cat'].apply(lambda x: -1 if x == 's_neg' or x == 'w_neg' else (1 if x == 's_pos' else 0))
        elif prefix == 'wn':
            df['participants_prediction'] = df['participants_polarity_5_cat'].apply(lambda x: -1 if x == 'w_pos' or x == 's_pos' else (1 if x == 'w_neg' else 0))
            df['psychologists_prediction'] = df['manual_polarity_5_cat'].apply(lambda x: -1 if x == 'w_pos' or x == 's_pos' else (1 if x == 'w_neg' else 0))
        elif prefix == 'sn':
            df['participants_prediction'] = df['participants_polarity_5_cat'].apply(lambda x: -1 if x == 's_pos' or x == 'w_pos' else (1 if x == 's_neg' else 0))
            df['psychologists_prediction'] = df['manual_polarity_5_cat'].apply(lambda x: -1 if x == 's_pos' or x == 'w_pos' else (1 if x == 's_neg' else 0))
    
    df['accuracy'] = df.apply(lambda x: 1 if x['participants_prediction'] == x['psychologists_prediction'] else 0, axis=1)

    df['conf_matrix'] = df.apply(lambda x: conf_matrix(x, 'psychologists_prediction', 'participants_prediction'), axis=1)

    list_precision = []

    conf_vals = df['conf_matrix'].value_counts().to_dict()
    
    if 'FP' not in conf_vals:
        conf_vals['FP'] = 0
    if 'TP' not in conf_vals:
        conf_vals['TP'] = 0
    if 'FN' not in conf_vals:
        conf_vals['FN'] = 0    
    if 'TN' not in conf_vals:
        conf_vals['TN'] = 0
        
    print(conf_vals)

    accuracy = (conf_vals['TP'] + conf_vals['TN']) / (conf_vals['TP'] + conf_vals['TN'] + conf_vals['FP'] + conf_vals['FN'])
    precision_pos = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FP'])

    # ------------------------------------------------------
    # verificar se devo incluir os NEUTROS também nas contas
    # ------------------------------------------------------

    recall_pos = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FN'])
    f1_score_pos = 2*(precision_pos*recall_pos / (precision_pos + recall_pos))

    precision_neg = conf_vals['TN'] / (conf_vals['TN'] + conf_vals['FN'])
    recall_neg = conf_vals['TN'] / (conf_vals['TN'] + conf_vals['FP'])
    f1_score_neg = 2*(precision_neg*recall_neg / (precision_neg + recall_neg))

    d = {
        'Accuracy': accuracy,
        'Precision - Positive': precision_pos,
        'Recall - Positive': recall_pos,
        'F1-Score - Positive': f1_score_pos,
        'Precision - Negative': precision_neg,
        'Recall - Negative': recall_neg,
        'F1-Score - Negative': f1_score_neg,
        'F1-Score - Average': (f1_score_pos + f1_score_neg) / 2
    }
       
    return pd.DataFrame.from_records([d])

In [311]:
df_for_analysis = create_psycho_versus_participants_df(df_data_psi_3_cat, 3)
print(len(df_for_analysis))
pd_stats_psy_part = perform_statistics_between_psycho_participants(df_for_analysis, 3, '')
pd_stats_psy_part

ids not found:  72
304
{0: 127, 'TP': 97, 'TN': 58, 'FP': 16, 'FN': 6}


,Accuracy,Precision - Positive,Recall - Positive,F1-Score - Positive,Precision - Negative,Recall - Negative,F1-Score - Negative,F1-Score - Average
0,0.875706,0.858407,0.941748,0.898148,0.90625,0.783784,0.84058,0.869364


In [312]:
df_for_analysis = create_psycho_versus_participants_df(df_data_psi_5_cat, 5)
print(len(df_for_analysis))
pd_stats_psy_part = perform_statistics_between_psycho_participants(df_for_analysis, 5, 'wp')
pd_stats_psy_part

ids not found:  114
262
{0: 169, 'TN': 51, 'TP': 26, 'FP': 11, 'FN': 5}


,Accuracy,Precision - Positive,Recall - Positive,F1-Score - Positive,Precision - Negative,Recall - Negative,F1-Score - Negative,F1-Score - Average
0,0.827957,0.702703,0.83871,0.764706,0.910714,0.822581,0.864407,0.814556


In [313]:
df_for_analysis = create_psycho_versus_participants_df(df_data_psi_5_cat, 5)
print(len(df_for_analysis))
pd_stats_psy_part = perform_statistics_between_psycho_participants(df_for_analysis, 5, 'sp')
pd_stats_psy_part

ids not found:  114
262
{0: 195, 'TN': 51, 'TP': 12, 'FP': 4, 'FN': 0}


,Accuracy,Precision - Positive,Recall - Positive,F1-Score - Positive,Precision - Negative,Recall - Negative,F1-Score - Negative,F1-Score - Average
0,0.940299,0.75,1.0,0.857143,1.0,0.927273,0.962264,0.909704


In [314]:
df_for_analysis = create_psycho_versus_participants_df(df_data_psi_5_cat, 5)
print(len(df_for_analysis))
pd_stats_psy_part = perform_statistics_between_psycho_participants(df_for_analysis, 5, 'wn')
pd_stats_psy_part

ids not found:  114
262
{0: 145, 'TN': 73, 'TP': 26, 'FN': 13, 'FP': 5}


,Accuracy,Precision - Positive,Recall - Positive,F1-Score - Positive,Precision - Negative,Recall - Negative,F1-Score - Negative,F1-Score - Average
0,0.846154,0.83871,0.666667,0.742857,0.848837,0.935897,0.890244,0.816551


In [315]:
df_for_analysis = create_psycho_versus_participants_df(df_data_psi_5_cat, 5)
print(len(df_for_analysis))
pd_stats_psy_part = perform_statistics_between_psycho_participants(df_for_analysis, 5, 'sn')
pd_stats_psy_part

ids not found:  114
262
{0: 178, 'TN': 73, 'TP': 9, 'FN': 2, 'FP': 0}


,Accuracy,Precision - Positive,Recall - Positive,F1-Score - Positive,Precision - Negative,Recall - Negative,F1-Score - Negative,F1-Score - Average
0,0.97619,1.0,0.818182,0.9,0.973333,1.0,0.986486,0.943243


In [316]:
df_data_psi_5_cat.head()

,id,Class_PSI1_5_cat,Class_PSI2_5_cat,Class_PSI3_5_cat,Class_PSI1_3_cat,Class_PSI2_3_cat,Class_PSI3_3_cat,manual_polarity_3_cat,manual_polarity_5_cat
1,102895,w_neg,s_neg,s_neg,neg,neg,neg,neg,s_neg
2,103085,s_neg,s_neg,s_neg,neg,neg,neg,neg,s_neg
3,103092,w_pos,w_pos,w_pos,pos,pos,pos,pos,w_pos
8,103546,s_pos,s_pos,w_pos,pos,pos,pos,pos,s_pos
10,103823,w_neg,w_neg,neu,neg,neg,neu,neg,w_neg


In [317]:
df_for_analysis.head()

,id,manual_polarity_5_cat,participants_polarity_5_cat,participants_prediction,psychologists_prediction,accuracy,conf_matrix
0,179689,w_pos,s_pos,-1,-1,1,TN
1,179701,neu,w_pos,-1,0,0,0
2,179748,s_neg,s_neg,1,1,1,TP
3,179886,w_pos,s_pos,-1,-1,1,TN
4,179917,w_neg,w_neg,0,0,1,0


In [318]:
df_for_analysis = create_psycho_versus_participants_df(df_data_psi_5_cat, 5)

# REMOVENDO OS NEUTROS
df_for_analysis.drop(df_for_analysis.loc[df_for_analysis['manual_polarity_5_cat']=='neu'].index, inplace=True)
df_for_analysis.drop(df_for_analysis.loc[df_for_analysis['participants_polarity_5_cat']=='neu'].index, inplace=True)

test = df_for_analysis['participants_polarity_5_cat']
pred = df_for_analysis['manual_polarity_5_cat']
confusion = confusion_matrix(test, pred)
print('Confusion Matrix\n')
print(confusion)

print('\nAccuracy: {:.6f}\n'.format(accuracy_score(test, pred)))

print('Micro Precision: {:.6f}'.format(precision_score(test, pred, average='micro')))
print('Micro Recall: {:.6f}'.format(recall_score(test, pred, average='micro')))
print('Micro F1-score: {:.6f}\n'.format(f1_score(test, pred, average='micro')))

print('Macro Precision: {:.6f}'.format(precision_score(test, pred, average='macro')))
print('Macro Recall: {:.6f}'.format(recall_score(test, pred, average='macro')))
print('Macro F1-score: {:.6f}\n'.format(f1_score(test, pred, average='macro')))

print('Weighted Precision: {:.6f}'.format(precision_score(test, pred, average='weighted')))
print('Weighted Recall: {:.6f}'.format(recall_score(test, pred, average='weighted')))
print('Weighted F1-score: {:.6f}'.format(f1_score(test, pred, average='weighted')))

print('\nClassification Report\n')
print(classification_report(test, pred, digits=6))

ids not found:  114
Confusion Matrix

[[ 9  0  9  0]
 [ 1 12  3 24]
 [ 7  0 26  5]
 [ 1 11 10 26]]

Accuracy: 0.506944

Micro Precision: 0.506944
Micro Recall: 0.506944
Micro F1-score: 0.506944

Macro Precision: 0.509033
Macro Recall: 0.506469
Macro F1-score: 0.497614

Weighted Precision: 0.507943
Weighted Recall: 0.506944
Weighted F1-score: 0.496166

Classification Report

              precision    recall  f1-score   support

       s_neg   0.500000  0.500000  0.500000        18
       s_pos   0.521739  0.300000  0.380952        40
       w_neg   0.541667  0.684211  0.604651        38
       w_pos   0.472727  0.541667  0.504854        48

    accuracy                       0.506944       144
   macro avg   0.509033  0.506469  0.497614       144
weighted avg   0.507943  0.506944  0.496166       144



In [319]:
df_for_analysis = create_psycho_versus_participants_df(df_data_psi_3_cat, 3)

# REMOVENDO OS NEUTROS
df_for_analysis.drop(df_for_analysis.loc[df_for_analysis['manual_polarity_3_cat']=='neu'].index, inplace=True)
df_for_analysis.drop(df_for_analysis.loc[df_for_analysis['participants_polarity_3_cat']=='neu'].index, inplace=True)

test = df_for_analysis['participants_polarity_3_cat']
pred = df_for_analysis['manual_polarity_3_cat']
confusion = confusion_matrix(test, pred)
print('Confusion Matrix\n')
print(confusion)

print('\nAccuracy: {:.6f}\n'.format(accuracy_score(test, pred)))

print('Micro Precision: {:.6f}'.format(precision_score(test, pred, average='micro')))
print('Micro Recall: {:.6f}'.format(recall_score(test, pred, average='micro')))
print('Micro F1-score: {:.6f}\n'.format(f1_score(test, pred, average='micro')))

print('Macro Precision: {:.6f}'.format(precision_score(test, pred, average='macro')))
print('Macro Recall: {:.6f}'.format(recall_score(test, pred, average='macro')))
print('Macro F1-score: {:.6f}\n'.format(f1_score(test, pred, average='macro')))

print('Weighted Precision: {:.6f}'.format(precision_score(test, pred, average='weighted')))
print('Weighted Recall: {:.6f}'.format(recall_score(test, pred, average='weighted')))
print('Weighted F1-score: {:.6f}'.format(f1_score(test, pred, average='weighted')))

print('\nClassification Report\n')
print(classification_report(test, pred, digits=6))

ids not found:  72
Confusion Matrix

[[58  6]
 [16 97]]

Accuracy: 0.875706

Micro Precision: 0.875706
Micro Recall: 0.875706
Micro F1-score: 0.875706

Macro Precision: 0.862766
Macro Recall: 0.882329
Macro F1-score: 0.869364

Weighted Precision: 0.884631
Weighted Recall: 0.875706
Weighted F1-score: 0.877332

Classification Report

              precision    recall  f1-score   support

         neg   0.783784  0.906250  0.840580        64
         pos   0.941748  0.858407  0.898148       113

    accuracy                       0.875706       177
   macro avg   0.862766  0.882329  0.869364       177
weighted avg   0.884631  0.875706  0.877332       177

